# Image Classifier

Simple Sweet and Short Image Classifier

This code is based on code from [fastai course](https://github.com/fastai/course22)



## Disclaimer

Uses DDGS for search. This library is not affiliated with DuckDuckGo and is for educational purposes only. It is not intended for commercial use or any purpose that violates DuckDuckGo's Terms of Service. [more here](https://pypi.org/project/duckduckgo-search/#disclaimer)

## Pre-requisite

In [ ]:
#skip if duckduckgo_search already installed
!pip install -Uqq duckduckgo_search

### Proxy

If you are getting rate limited in DDGS, as per docs you can use proxy. Below implementation uses tor.
More info here https://pypi.org/project/duckduckgo-search/#proxy

## Code

### Utilities

In [ ]:
## Search images using API (here duckduckgo)

from time import sleep
from fastcore.all import *
from duckduckgo_search import DDGS

def search_images(term, max_images=200, retries=3, delay=2):
    for i in range(retries):
        try:
            with DDGS(proxy="socks5://127.0.0.1:9050") as ddgs:

                return L(ddgs.images(keywords=term, max_results=max_images)).itemgot('image')
        except Exception as e:
            if i == retries - 1: raise e
            print(f"Rate limited, retrying in {delay} seconds...")
            sleep(delay)
            delay *= 2  # Exponential backoff

### Code for Bird Classifier from the Course

In [ ]:
urls = search_images("bird photos", max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0],dest,show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('forest photos', max_images=1)[0],'forest.jpg',show_progress=False)
Image.open('forest.jpg').to_thumb(256,256)

In [ ]:
searches = 'forest', 'bird'
path = Path('bird_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest,urls=search_images(f'{o} photo'))
    sleep(10)
    download_images(dest,urls=search_images(f'{o} sun photo'))
    sleep(10)
    download_images(dest,urls=search_images(f'{o} shade photo'))
    sleep(10)
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
# verify images

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

for above code

`get_image_files(path):`

Source: FastAI Library.
Purpose: This function is part of the FastAI library. It's used to get a list of all image files in a directory (recursively including subdirectories). It helps in gathering image data for machine learning tasks.


`verify_images(...):`

Source: FastAI Library.
Purpose: Also part of the FastAI library, this function checks if images in the provided list (from get_image_files) can be opened. It's useful for filtering out corrupt or unreadable image files before processing or training a model.

`Path.unlink:`

Source: Standard Python Library (pathlib module).
Purpose: Path is a class from Python’s pathlib module, and unlink is a method of the Path class. It is used to remove (or delete) a file at the given path. It’s similar to the os.remove function in the os module but is a part of the object-oriented approach provided by pathlib.

`map(...):`

Source: Standard Python functionality.
Purpose: map is a built-in Python function that applies a given function to each item of an iterable (like a list) and returns a map object (which is an iterator). In your code, map is used to apply the Path.unlink method to each path in the failed list.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192,method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
predicted_category,_,probs= learn.predict(PILImage.create('bird.jpg'))
print(f"This is a: {predicted_category}")
print(f"Probability it's a bird: {probs[0]:.4f}")

In [ ]:
# export model
learn.export('birdclassifier.pkl')

In [ ]:
# import model
learn = load_learner('birdclassifier.pkl')

In [ ]:
category_prediction,_,probs= learn.predict(PILImage.create('forest.jpg'))

print(f"This is a: {category_prediction}")

labels= learn.dls.vocab
print(f"{labels}")

for i, label in enumerate(labels):
    print(f"Probability it's a {label}: {probs[i]:.4f}")

### Code for Jedi / sith Classifier

In [ ]:
searches = 'jedi', 'sith'
path = Path('jedi_or_sith')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest,urls=search_images(f'{o} photo'))
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
# verify images

failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192,method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

from PIL import Image
import io



# Create a file upload widget
upload_button = widgets.FileUpload(
    description='Upload Image',
    accept='.jpg, .jpeg, .png',  # Accepted file types
    multiple=False  # Allow only one file
)

# Create an output widget to display the result
output = widgets.Output()

def on_upload_change(change):
    """Callback function when an image is uploaded."""
    with output:
        clear_output()  # Clear previous output
        if upload_button.value:  # Check if a file is uploaded
            # Get the uploaded file
            uploaded_file = list(upload_button.value.values())[0]
            # Read the image data
            image_data = uploaded_file['content']
            # Convert to PIL image
            img = PILImage.create(io.BytesIO(image_data))
            # Display the image
            display(img.to_thumb(224, 224))  # Resize for display
            # Make prediction
            predicted_category, _, probs = learn.predict(img)
            # Show the result
            print(f"This is a: {predicted_category}")

# Attach the callback to the upload widget
upload_button.observe(on_upload_change, names='value')

# Display the upload button and output widget
display(upload_button)
display(output)

In [ ]:
# export model
learn.export('jedi_or_sith_classifier.pkl')